In [188]:
import pandas as pd
import datetime
import plotly.express as px
from datetime import datetime


In [2]:
geo = pd.read_csv("cities_lat_lon.csv")
df = pd.read_csv("cities_weather.csv")
df.head()

,Unnamed: 0,cod,message,cnt,list,city
0,0,200,0,40,"[{'dt': 1681462800, 'main': {'temp': 8.19, 'fe...","{'id': 6435453, 'name': 'Huisnes-sur-Mer', 'co..."
1,1,200,0,40,"[{'dt': 1681462800, 'main': {'temp': 8.61, 'fe...","{'id': 2978640, 'name': 'St-Malo', 'coord': {'..."
2,2,200,0,40,"[{'dt': 1681462800, 'main': {'temp': 6.6, 'fee...","{'id': 3034483, 'name': 'Bayeux', 'coord': {'l..."
3,3,200,0,40,"[{'dt': 1681462800, 'main': {'temp': 7.33, 'fe...","{'id': 3003796, 'name': 'Le Havre', 'coord': {..."
4,4,200,0,40,"[{'dt': 1681462800, 'main': {'temp': 7.16, 'fe...","{'id': 2982652, 'name': 'Rouen', 'coord': {'la..."


In [25]:
df['city'] = df['city'].apply(eval)
df['list'] = df['list'].apply(eval)

In [192]:
w = pd.DataFrame()
w.loc[0, 0] = 0

for i in range(df.shape[0]):
    k = w.index[-1]
    for j in range(len(df.loc[i, 'list'])) : 
        w.loc[i+k+j, 'city_id'] = int(i+1)
        w.loc[i+k+j, 'city'] = df.loc[i, 'city']['name']
        #avec j l'index qui permet d'accéder aux 5 différents jours
        w.loc[i+k+j, 'date'] = df.loc[i, 'list'][j]['dt']
        w.loc[i+k+j, 'temperature'] = df.loc[i, 'list'][j]['main']['temp']
        w.loc[i+k+j, 'real_feel'] = df.loc[i, 'list'][j]['main']['feels_like']
        w.loc[i+k+j, 'humidity'] = df.loc[i, 'list'][j]['main']['humidity']
        w.loc[i+k+j, 'weather_description'] = df.loc[i, 'list'][j]['weather'][0]['description']
        w.loc[i+k+j, 'wind_speed'] = df.loc[i, 'list'][j]['wind']['speed']
    

w['day'] = w['date'].apply(lambda x : datetime.utcfromtimestamp(x).strftime('%Y-%m-%d'))
w['time'] = w['date'].apply(lambda x : datetime.utcfromtimestamp(x).strftime('%H:%M:%S'))

w = w.iloc[:, [0, 1, 2, 9, 10, 3, 4, 5, 6, 7, 8]]
w = w.drop ([0, 'date'], axis=1) 
w



,city_id,city,day,time,temperature,real_feel,humidity,weather_description,wind_speed
0,1.0,Huisnes-sur-Mer,2023-04-14,09:00:00,8.19,4.19,95.0,light rain,8.48
1,1.0,Huisnes-sur-Mer,2023-04-14,12:00:00,9.48,6.86,92.0,moderate rain,5.22
2,1.0,Huisnes-sur-Mer,2023-04-14,15:00:00,12.15,11.64,85.0,light rain,4.49
3,1.0,Huisnes-sur-Mer,2023-04-14,18:00:00,10.59,10.16,94.0,light rain,5.22
4,1.0,Huisnes-sur-Mer,2023-04-14,21:00:00,9.35,5.46,79.0,broken clouds,9.50
...,...,...,...,...,...,...,...,...,...
1956,35.0,La Rochelle,2023-04-18,18:00:00,12.83,12.07,73.0,light rain,7.91
1957,35.0,La Rochelle,2023-04-18,21:00:00,10.53,9.62,76.0,clear sky,8.50
1958,35.0,La Rochelle,2023-04-19,00:00:00,9.33,5.94,76.0,clear sky,7.48
1959,35.0,La Rochelle,2023-04-19,03:00:00,8.12,4.39,79.0,clear sky,7.45


In [215]:
avg_city_w = w.groupby(['city', 'day'])['temperature', 'real_feel', 'humidity'].mean().reset_index()
a = pd.DataFrame(avg_city_w)
a

C:\Users\Celine\AppData\Local\Temp\ipykernel_13852\2158572186.py:1: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



,city,day,temperature,real_feel,humidity
0,Aigues-Mortes,2023-04-14,12.204000,11.212000,66.400000
1,Aigues-Mortes,2023-04-15,13.980000,12.806250,52.625000
2,Aigues-Mortes,2023-04-16,14.471250,13.285000,50.250000
3,Aigues-Mortes,2023-04-17,14.622500,13.503750,52.125000
4,Aigues-Mortes,2023-04-18,12.672500,11.820000,69.875000
...,...,...,...,...,...
205,Vieux Lyon,2023-04-15,7.907500,5.571250,91.750000
206,Vieux Lyon,2023-04-16,9.373750,7.755000,76.375000
207,Vieux Lyon,2023-04-17,9.601250,7.873750,79.375000
208,Vieux Lyon,2023-04-18,8.641250,7.380000,80.375000


In [203]:
from plotly.offline import plot
import plotly.io as pio

fig = px.scatter(a, x = 'city', y = 'temperature', animation_frame = "day")
plot(fig)

pio.write_html(fig, "20230414_20230419_temperature_forecast.html")

In [204]:
fig = px.scatter(a, x = 'city', y = 'humidity', animation_frame = "day")
plot(fig)

pio.write_html(fig, "20230414_20230419_humidity_forecast.html")

In [201]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x= a['city'], y=a['temperature'], name="T(°C)", mode = 'markers'
),
    secondary_y=False)

fig.add_trace(
    go.Scatter(x=a['city'], y=a['humidity'], name="%", mode = 'markers'),
    secondary_y=True
)

# Add figure title
fig.update_layout(
    title_text="Température et humidité moyenne"
)

# Set x-axis title
fig.update_xaxes(title_text="City")

# Set y-axes titles
fig.update_yaxes(title_text="<b>primary</b> yaxis title", secondary_y=False)
fig.update_yaxes(title_text="<b>secondary</b> yaxis title", secondary_y=True)

fig.show()

In [207]:
#détermination des destinations avec la meilleur météo : meilleure température et moins d'humidité
a.sort_values(by=["temperature", "humidity"], ascending=[False, True]).head(10)

,city,day,temperature,humidity
141,Marseille,2023-04-17,14.89875,50.500
69,Collioure,2023-04-17,14.71500,51.000
3,Aigues-Mortes,2023-04-17,14.62250,52.125
63,Cassis,2023-04-17,14.53250,56.375
2,Aigues-Mortes,2023-04-16,14.47125,50.250
67,Collioure,2023-04-15,14.31375,57.250
140,Marseille,2023-04-16,14.19500,52.750
176,Saintes-Maries-de-la-Mer,2023-04-16,14.13250,51.625
62,Cassis,2023-04-16,14.06625,54.875
68,Collioure,2023-04-16,14.04000,54.875


In [216]:
top = a.sort_values(by=["real_feel", "humidity"], ascending=[False, True]).head(10)
top_city = top['city'].drop_duplicates()
top_city

141                   Marseille
69                    Collioure
63                       Cassis
3                 Aigues-Mortes
176    Saintes-Maries-de-la-Mer
Name: city, dtype: object

In [217]:
w.to_csv("20230414_20230419_whole_weather_forecast.csv")
a.to_csv("20230414_20230419_avg_day_weather_forecast.csv")

In [219]:
gps = pd.read_csv("cities_lat_lon.csv")
gps.columns

Index(['Unnamed: 0', 'cities', 'latitudes', 'longitudes'], dtype='object')

In [220]:
a_gps = a.merge(gps, left_on = 'city', right_on='cities')

In [230]:
#Map of temperature of cities
fig = px.scatter_mapbox(a_gps, lat="latitudes", lon="longitudes", animation_frame = "day", color="temperature",color_continuous_scale='turbo', size="humidity", zoom=3, mapbox_style="carto-positron", hover_name="city")
fig.show()